In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

In [ ]:
# 显示cell运行时长
%load_ext klab-autotime

In [105]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/kesci/nltk_data...


KeyboardInterrupt: 

In [1]:
pip install nltk

     |████████████████████████████████| 1.4MB 138kB/s eta 0:00:01
     |████████████████████████████████| 686kB 135kB/s eta 0:00:01
Failed to build nltk
  Running setup.py install for nltk ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchtext 

     |████████████████████████████████| 71kB 208kB/s eta 0:00:011
     |████████████████████████████████| 1.0MB 61kB/s eta 0:00:014
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
import torch
from torch.nn import functional as F
import numpy as np
from torchtext import data
from torchtext import datasets
import torchtext.vocab as Vocab
from torchtext.vocab import Vectors, GloVe
import torch.utils.data as Data
from torch import nn
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import collections
import time

In [1]:
import pandas as pd
path = r'/home/kesci/input/qiu_assignment74907/train.tsv'

In [2]:
train_data = pd.read_csv(path,sep = '\t')

In [3]:
train_data.head(6)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2


In [10]:
train_data.Phrase[38:56]

38    some of which occasionally amuses but none of ...
39                                        some of which
40                                                 some
41                                             of which
42                                                which
43    occasionally amuses but none of which amounts ...
44                                         occasionally
45    amuses but none of which amounts to much of a ...
46                                               amuses
47         but none of which amounts to much of a story
48                                                  but
49             none of which amounts to much of a story
50                                                 none
51                  of which amounts to much of a story
52                     which amounts to much of a story
53                           amounts to much of a story
54                                              amounts
55                                   to much of 

### 一、描述性分析

In [84]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
PhraseId      156060 non-null int64
SentenceId    156060 non-null int64
Phrase        156060 non-null object
Sentiment     156060 non-null int64
dtypes: int64(3), object(1)
memory usage: 4.8+ MB


In [85]:
train_data.Sentiment.value_counts()

2    79582
3    32927
1    27273
4     9206
0     7072
Name: Sentiment, dtype: int64

In [86]:
each_len = [len(x) for x in train_data.Phrase]

In [87]:
average_len = int(sum(each_len)/len(each_len))
print('Max len is {} and average len is {}'.format(max(each_len),average_len))

Max len is 283 and average len is 40


### 二、数据预处理

In [88]:
def clean_sentences(df):
    reviews = []
    for sent in tqdm(df['Phrase']):
        #remove html content
        review_text = BeautifulSoup(sent).get_text()
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        
        #tokenize the sentences
        words = word_tokenize(review_text.lower())
        
        #lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)

In [102]:
train_sentences = clean_sentences(train_data)
print(len(train_sentences))

  0%|          | 1/156060 [00:02<105:54:18,  2.44s/it]/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:294: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
100%|██████████| 156060/156060 [00:32<00:00, 4774.06it/s]

156060


In [103]:
train_sentences[20:26]

[['is'],
 ['good', 'for', 'the', 'goose'],
 ['good'],
 ['for', 'the', 'goose'],
 ['for'],
 ['the', 'goose']]

In [104]:
senten_len = [len(x) for x in train_sentences]
ave_len = int(sum(senten_len)/len(senten_len))
print('Max len is {} and average len is {}'.format(max(senten_len),ave_len))

Max len is 48 and average len is 6


In [105]:
preprocess_data = pd.DataFrame()

In [106]:
preprocess_data.insert(0,'Phrases',train_sentences)
preprocess_data.insert(1,'Sentiment',train_data['Sentiment'])

In [107]:
preprocess_data.head(6)

,Phrases,Sentiment
0,"[a, series, of, escapade, demonstrating, the, ...",1
1,"[a, series, of, escapade, demonstrating, the, ...",2
2,"[a, series]",2
3,[a],2
4,[series],2
5,"[of, escapade, demonstrating, the, adage, that...",2


In [108]:
preprocess_data.to_csv('preprocess_data')

In [109]:
textcnn参考https://www.cnblogs.com/mxp-neu/articles/9949409.html
因为最后一层是max_pooling所以padding对其没有影响

mask问题https://www.zhihu.com/question/305508138/answer/552710027

SyntaxError: invalid syntax (<ipython-input-109-64e577fd40c0>, line 1)

In [4]:
train_data = pd.read_csv('/home/kesci/work/preprocess_data')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 3 columns):
Unnamed: 0    156060 non-null int64
Phrases       156060 non-null object
Sentiment     156060 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
df = train_data.drop(columns = 'Unnamed: 0')
reviews = []
for review in df.Phrases:
    reviews.append(eval(review))
reviews[20:26]

[['is'],
 ['good', 'for', 'the', 'goose'],
 ['good'],
 ['for', 'the', 'goose'],
 ['for'],
 ['the', 'goose']]

### 三、建立词向量

In [6]:
#建立词语索引，保留在数据集中至少出现5次的词
counter = collections.Counter([tk for st in reviews for tk in st])
vocab = Vocab.Vocab(counter, min_freq=5)

In [7]:
counter.most_common(10)

[('the', 51633),
 ('a', 45066),
 ('of', 32702),
 ('and', 32177),
 ('to', 22761),
 ('it', 18785),
 ('s', 17447),
 ('in', 14006),
 ('is', 13476),
 ('that', 12338)]

In [8]:
max_l = 48
def pad(x):
    return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))
features = torch.tensor([pad([vocab.stoi[word] for word in words]) 
                    for words in reviews])

In [9]:
reviews[2]

['a', 'series']

In [10]:
features[2]

tensor([  3, 338,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0])

In [11]:
labels = torch.tensor([train_data.Sentiment])

In [12]:
embeddings = {}
embeddings_file = '/home/kesci/input/qiu_assignment74907/glove.6B.50d.txt'
with open(embeddings_file, "r", encoding="utf8") as input_data:
    for line in input_data:
        line = line.split()
        try:
            float(line[1])
            word = line[0]
            embeddings[word] = line[1:]
        except ValueError:
            continue
len(embeddings['the'])

50

In [15]:
#本来直接torch.tensor(embeddings[word], dtype = torch.float)
#但是报错too many dimensions，不知如何解决，就先转换为np.array
embedding_matrix = torch.zeros((num_words, 50))
embedding_matrix[0]
a = np.array(embeddings['the'],dtype = float)
b = torch.tensor(a)

In [16]:
num_words = len(vocab.itos)
embedding_matrix = torch.zeros((num_words, 50))
oov_count = 0 # out of vocabulary
for i, word in enumerate(vocab.itos):
    if word in embeddings:
        word_embedding = np.array(embeddings[word],dtype = float)
        embedding_matrix[i] = torch.tensor(word_embedding, dtype=torch.float64)
    else:
        if word == '<pad>':
            print('pad')
            continue
        oov_count += 1
        embedding_matrix[i] = torch.randn(50)
print("There are %d oov words." % oov_count)

pad
There are 282 oov words.


In [17]:
#查看一下生成的词向量
embedding_matrix[2]

tensor([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
         1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
        -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
        -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
         4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
         7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
        -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
         1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01])

In [18]:
embeddings['the']

['0.418',
 '0.24968',
 '-0.41242',
 '0.1217',
 '0.34527',
 '-0.044457',
 '-0.49688',
 '-0.17862',
 '-0.00066023',
 '-0.6566',
 '0.27843',
 '-0.14767',
 '-0.55677',
 '0.14658',
 '-0.0095095',
 '0.011658',
 '0.10204',
 '-0.12792',
 '-0.8443',
 '-0.12181',
 '-0.016801',
 '-0.33279',
 '-0.1552',
 '-0.23131',
 '-0.19181',
 '-1.8823',
 '-0.76746',
 '0.099051',
 '-0.42125',
 '-0.19526',
 '4.0071',
 '-0.18594',
 '-0.52287',
 '-0.31681',
 '0.00059213',
 '0.0074449',
 '0.17778',
 '-0.15897',
 '0.012041',
 '-0.054223',
 '-0.29871',
 '-0.15749',
 '-0.34758',
 '-0.045637',
 '-0.44251',
 '0.18785',
 '0.0027849',
 '-0.18411',
 '-0.11514',
 '-0.78581']

### 四、建立模型

In [93]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers):
        '''
        @params:
            vocab: 在数据集上创建的词典，用于获取词典大小
            embed_size: 嵌入维度大小
            num_hiddens: 隐藏状态维度大小
            num_layers: 隐藏层个数
        '''
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        
        # encoder-decoder framework
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True)
        self.decoder = nn.Linear(4*num_hiddens, 5) # 初始时间步和最终时间步的隐藏状态作为全连接层输入
                                                   #总共有五类情感 
    def forward(self, inputs):
        '''
        @params:
            inputs: 词语下标序列，形状为 (batch_size, seq_len) 的整数张量
        @return:
            outs: 对文本情感的预测，形状为 (batch_size, 2) 的张量
        '''
        # 因为LSTM需要将序列长度(seq_len)作为第一维，所以需要将输入转置
        embeddings = self.embedding(inputs.permute(1, 0)) # (seq_len, batch_size, d)
        # rnn.LSTM 返回输出、隐藏状态和记忆单元，格式如 outputs, (h, c)
        outputs, _ = self.encoder(embeddings) # (seq_len, batch_size, 2*h)
        encoding = torch.cat((outputs[0], outputs[-1]), -1) # (batch_size, 4*h)
        outs = self.decoder(encoding) # (batch_size, 2)
        return outs

embed_size, num_hiddens, num_layers = 50, 100, 2
net = BiRNN(vocab, embed_size, num_hiddens, num_layers)

In [94]:
#将预训练好的词向量拷贝至embedding层
net.embedding.weight.data.copy_(embedding_matrix)

tensor([[ 0.2601,  0.1657, -0.1617,  ...,  0.4756, -0.8590,  0.3660],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4180,  0.2497, -0.4124,  ..., -0.1841, -0.1151, -0.7858],
        ...,
        [-0.8092, -0.0310,  0.5102,  ...,  0.0549, -0.1811,  1.4094],
        [-0.4244, -0.0948, -0.1946,  ...,  0.3926,  0.8929, -0.2771],
        [-1.6331,  0.3790, -1.1350,  ...,  1.6321, -1.1320,  0.4799]])

### 五、构建训练数据

In [21]:
labels = torch.tensor(df.Sentiment)

In [22]:
train_set = Data.TensorDataset(features, labels)

In [23]:
batch_size = 64
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
print('#batches:', len(train_iter))

X torch.Size([64, 48]) y torch.Size([64])
#batches: 2439


In [138]:
def train(train_iter,  net, loss, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y) 
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        print('epoch %d, loss %.4f, train acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, time.time() - start))


In [139]:
lr, num_epochs = 0.01, 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()

train(train_iter, net, loss, optimizer, device, num_epochs)

training on  cpu
epoch 1, loss 0.7402, train acc 0.695, time 535.0 sec
epoch 2, loss 0.3572, train acc 0.705, time 527.3 sec
epoch 3, loss 0.2312, train acc 0.713, time 642.4 sec
epoch 4, loss 0.1708, train acc 0.717, time 812.9 sec
epoch 5, loss 0.1349, train acc 0.721, time 1117.8 sec


In [142]:
def predict_sentiment(net, vocab, sentence):
    '''
    @params：
        net: 训练好的模型
        vocab: 在该数据集上创建的词典，用于将给定的单词序转换为单词下标的序列，从而输入模型
        sentence: 需要分析情感的文本，以单词序列的形式给出
    @return: 预测的结果，positive 为正面情绪文本，negative 为负面情绪文本
    '''
    device = list(net.parameters())[0].device # 读取模型所在的环境
    sentence = torch.tensor([vocab.stoi[word] for word in sentence], device=device)
    label = torch.argmax(net(sentence.view((1, -1))), dim=1)
    dict = {
        tensor([0]):'negative'
       tensor([1]):'somewhat negative'
        tensor([2]):'neutral'
        tensor([3]):'somewhat positive'
        tensor([4]):'positive'
        }
    return dict(label)

predict_sentiment(net, vocab, ['this', 'movie', 'is', 'so', 'terrible'])

tensor([0])

In [2]:
dict = {
        tensor[0]:'negative'
       tensor[1]:'somewhat negative'
        tensor[2]:'neutral'
        tensor[3]:'somewhat positive'
        tensor[4]:'positive'
        }

SyntaxError: invalid syntax (<ipython-input-2-7f9a532762e0>, line 3)

In [25]:
class TextCNN(nn.Module):
    def __init__(self, vocab, embed_size, kernel_sizes, num_channels):
        '''
        @params:
            vocab: 在数据集上创建的词典，用于获取词典大小
            embed_size: 嵌入维度大小
            kernel_sizes: 卷积核大小列表
            num_channels: 卷积通道数列表
        '''
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size) # 参与训练的嵌入层
        self.constant_embedding = nn.Embedding(len(vocab), embed_size) # 不参与训练的嵌入层
        
        self.pool = GlobalMaxPool1d() # 时序最大池化层没有权重，所以可以共用一个实例
        self.convs = nn.ModuleList()  # 创建多个一维卷积层
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(in_channels = 2*embed_size, 
                                        out_channels = c, 
                                        kernel_size = k))
            
        self.decoder = nn.Linear(sum(num_channels), 5)
        self.dropout = nn.Dropout(0.5) # 丢弃层用于防止过拟合

    def forward(self, inputs):
        '''
        @params:
            inputs: 词语下标序列，形状为 (batch_size, seq_len) 的整数张量
        @return:
            outputs: 对文本情感的预测，形状为 (batch_size, 2) 的张量
        '''
        embeddings = torch.cat((
            self.embedding(inputs), 
            self.constant_embedding(inputs)), dim=2) # (batch_size, seq_len, 2*embed_size)
        # 根据一维卷积层要求的输入格式，需要将张量进行转置
        embeddings = embeddings.permute(0, 2, 1) # (batch_size, 2*embed_size, seq_len)
        
        encoding = torch.cat([
            self.pool(F.relu(conv(embeddings))).squeeze(-1) for conv in self.convs], dim=1)
        # encoding = []
        # for conv in self.convs:
        #     out = conv(embeddings) # (batch_size, out_channels, seq_len-kernel_size+1)
        #     out = self.pool(F.relu(out)) # (batch_size, out_channels, 1)
        #     encoding.append(out.squeeze(-1)) # (batch_size, out_channels)
        # encoding = torch.cat(encoding) # (batch_size, out_channels_sum)
        
        # 应用丢弃法后使用全连接层得到输出
        outputs = self.decoder(self.dropout(encoding))
        return outputs
embed_size = 50    
net_2 = TextCNN(vocab, embed_size, num_hiddens, num_layers)


NameError: name 'num_hiddens' is not defined

In [ ]:
net_2.embedding.weight.data.copy_(embedding_matrix)

In [20]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, net, loss, optimizer, device, num_epochs)

NameError: name 'net' is not defined